In [1]:
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from sklearn import datasets
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import FactorAnalysis as FA

/Users/chikuwa/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/Users/chikuwa/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
xl_path = 'xxx.xlsx'
xl = pd.ExcelFile(xl_path) 
data = xl
    #sheet_namesメソッドでExcelブック内の各シートの名前をリストで取得できる
xl_sheet_name = xl.sheet_names
    #DataFrameとして一つ目のsheetを読込
df = xl.parse(xl_sheet_name[3])
    # データをコピーする。
x_df = df.copy()
y_df = df.copy()

    #属性データのみを取り出す
x_df = x_df.drop('FONT',axis=1)
    #目的変数のみを取り出す
drop_idx = ['声が大きい',  '柔らかい', '感情的', '力強い', '穏やかな', '大人っぽい', '賑やかな', 'はっきりした', '和やかな', '積極的な', '生き生きした', '暖かい', 'さっぱりした', '好意']
y_df = y_df.drop(drop_idx,axis=1)

df_columns = drop_idx

ValueError: labels ['声が大きい' '柔らかい' '感情的' '力強い' '穏やかな' '大人っぽい' '賑やかな' 'はっきりした' '和やかな' '積極的な'
 '生き生きした' '暖かい' 'さっぱりした' '好意'] not contained in axis

In [ ]:
def my_load_dataset(): 
    
    xl_path = 'xxx.xlsx'
    xl = pd.ExcelFile(xl_path) 
    xl_sheet_name = xl.sheet_names
    df = xl.parse(xl_sheet_name[1])
    x_df = df.copy()
    y_df = df.copy()
    drop_idx = ['A',  'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N']
    x_df = x_df.drop('FONT',axis=1)
    y = y_df.drop(drop_idx,axis=1)
    df_columns = drop_idx
    
    return x_df, y, df_columns 

In [15]:
def my_scaler(X, df_columns): 
    
    scaler = StandardScaler()
    x_sc = scaler.fit_transform(X)
    x_sc = pd.DataFrame(x_sc)
    x_sc.columns = df_columns
    
    return x_sc

In [16]:
def my_fa(X_sc, Y, n): 
    
    fa = FA(n_components=n, max_iter=500)
    fa.fit(X_sc)
    print("fa.components_")
    print(fa.components_)
    
    x_fa = fa.fit_transform(X_sc)

    x_data = x_fa[:,0]
    y_data = x_fa[:,1]
    fa0 = fa.components_[0]
    fa1 = fa.components_[1]
    
    plt.figure(figsize=(9,9))
    plt.scatter(x_data, y_data, c="blue", marker=".")
    for i in range(fa0.shape[0]):
        plt.arrow(0, 0, fa0[i]*1.5, fa1[i]*1.5, color='r', width=0.0005, head_width=0.07, alpha=0.8)
        plt.text(fa0[i]*1.8, fa1[i]*1.8, X_sc.columns.values[i], color='r', fontsize = 18, fontname="MS Gothic")
    plt.xlim([-2.5,2.5])
    plt.ylim([-2.5,2.5])
    plt.xlabel('Factor 1', fontsize = 20)
    plt.ylabel('Factor 2', fontsize = 20)
    plt.title(str(n)+" factor", fontsize = 20)
    plt.tick_params(labelsize = 18)
    plt.grid()
    c = patches.Circle(xy=(0, 0), radius=1, ec='r', fill=False)
    plt.axes().add_patch(c)
    #plt.show()
    plt.savefig("FA_" + str('%02g' %n) + '.png')

In [17]:
def my_regression(X, Y, mytitle='Ref', mylabel='Multiple regression analysis'): 
    
    clf = linear_model.LinearRegression(fit_intercept=True, normalize=False, copy_X=True, n_jobs=1)
    clf.fit(X, Y)
    print(pd.DataFrame({"Name":X.columns, "Coefficients":clf.coef_}).sort_values(by='Coefficients')) # 係数でソート
    print(clf.intercept_) # 切片
    print(clf.score(X,Y)) # R^2

    plt.figure()
    plt.title(mytitle)
    plt.xlabel('RM (number of rooms)', fontsize=14)
    plt.ylabel('PRICE (target)', fontsize=14)
    plt.scatter(X["RM"], Y, c='blue', label='Raw data')
    plt.scatter(X.RM, clf.predict(X), c='red', label=mylabel)
    plt.legend(loc='lower right', fontsize=12)
    plt.show()

In [18]:
if __name__ == '__main__':
    
    # データセットを読み込み次のの3つを返す。X, Y, 説明変数Xのカラム(列)名
    data = my_load_dataset()
    X = data[0]
    Y = data[1]
    df_columns = data[2]
    print(X)
    
    # 説明変数Xを標準化
    X_sc = my_scaler(X, df_columns)
    print(X_sc)
    
    # 因子分析
    mycheck_Num=[2,3,12,13]
    for i in mycheck_Num:
        print(i)
        my_fa(X_sc, Y, i)
    
    # 説明変数Xを減らす
    # 　因子分析の結果から独自判断で下記4つの説明変数を削除
   # drop_col_1 = ['LSTAT', 'INDUS', 'NOX', 'AGE']
    #X1 = X.drop(drop_col_1, axis=1) # X1は9つの説明変数Xで構成されるデータフレーム(pandas形式)

    # 線形重回帰分析
    #my_regression(X, Y, mytitle='Ref', mylabel='13feature analysis')  # 1.説明変数X全13因子を使用しての回帰
    #my_regression(X1, Y, mytitle='After FA', mylabel='9feature analysis') # 2.説明変数X9因子を使用しての回帰

KeyError: "['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N'] not found in axis"